# 1 初始化

## 1.1 导入包

In [1]:
import lightgbm as lgb
import numpy as np
import os
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from tqdm import tqdm
import logging

log_fmt = "[%(asctime)s] %(levelname)s in %(module)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)

import warnings
warnings.filterwarnings('ignore')

os.environ["CUDA_VISIBLE_DEVICES"] = "3"

## 1.2 超参数定义

In [2]:
base_path = '/home/zengrui/datasets/190829_Kanshan_zjfx'

def save_param(param, param_name):
    with open(f'{base_path}/temp/{param_name}.txt', 'wb') as f:
        pickle.dump(param, f)
        
def read_param(param_name):
    with open(f'{base_path}/temp/{param_name}.txt', 'rb') as f:
        return pickle.load(f)

## 1.3 函数定义

In [3]:
def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))


def extract_hour(s):
    return s.apply(lambda x: int(x.split('-')[1][1:]))

# 2 加载、处理数据

## 2.1 加载邀请回答数据

In [4]:
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
logging.info("invite %s", train.shape)

test = pd.read_csv(f'{base_path}/invite_info_evaluate_1_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']
logging.info("test %s", test.shape)

sub = test.copy()

sub_size = len(sub)

train['day'] = extract_day(train['dt'])
train['hour'] = extract_hour(train['dt'])

test['day'] = extract_day(test['dt'])
test['hour'] = extract_hour(test['dt'])

del train['dt'], test['dt']

[2019-12-09 08:38:40,642] INFO in <ipython-input-4-7a2c6478cb9e>: invite (9489162, 4)
[2019-12-09 08:38:41,753] INFO in <ipython-input-4-7a2c6478cb9e>: test (1141683, 3)


## 2.2 加载问题

丢弃了**问题info**的字词编码。

In [5]:
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t')
ques.columns = ['qid', 'q_dt', 'title_t1', 'title_t2', 'desc_t1', 'desc_t2', 'topic']
del ques['title_t1'], ques['title_t2'], ques['desc_t1'], ques['desc_t2']
logging.info("ques %s", ques.shape)

ques['q_day'] = extract_day(ques['q_dt'])
ques['q_hour'] = extract_hour(ques['q_dt'])
del ques['q_dt']

[2019-12-09 08:39:11,860] INFO in <ipython-input-5-958902b92fc5>: ques (1829900, 3)


## 2.3 加载回答

丢弃了**回答info**的字词编码，并与**问题info**合并。

In [6]:
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t')
ans.columns = ['aid', 'qid', 'uid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 'has_img',
               'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
               'reci_xxx', 'reci_no_help', 'reci_dis']
del ans['ans_t1'], ans['ans_t2']
logging.info("ans %s", ans.shape)

ans['a_day'] = extract_day(ans['ans_dt'])
ans['a_hour'] = extract_hour(ans['ans_dt'])
del ans['ans_dt']

ans = pd.merge(ans, ques, on='qid')
del ques

[2019-12-09 08:40:20,275] INFO in <ipython-input-6-935311f5f359>: ans (4513735, 18)


## 2.4 处理时间数据

回答距提问的天数。

In [7]:
ans['diff_qa_days'] = ans['a_day'] - ans['q_day']

时间窗口划分。

In [8]:
train_start = 3838
train_end = 3867

val_start = 3868
val_end = 3874

label_end = 3867
label_start = label_end - 6

train_label_feature_end = label_end - 7
train_label_feature_start = train_label_feature_end - 22

train_ans_feature_end = label_end - 7
train_ans_feature_start = train_ans_feature_end - 50

val_label_feature_end = val_start - 1
val_label_feature_start = val_label_feature_end - 22

val_ans_feature_end = val_start - 1
val_ans_feature_start = val_ans_feature_end - 50

train_label_feature = train[(train['day'] >= train_label_feature_start) & (train['day'] <= train_label_feature_end)]
logging.info("train_label_feature %s", train_label_feature.shape)

val_label_feature = train[(train['day'] >= val_label_feature_start) & (train['day'] <= val_label_feature_end)]
logging.info("val_label_feature %s", val_label_feature.shape)

train_label = train[(train['day'] > train_label_feature_end)]

logging.info("train feature start %s end %s, label start %s end %s", train_label_feature['day'].min(),
             train_label_feature['day'].max(), train_label['day'].min(), train_label['day'].max())

logging.info("test feature start %s end %s, label start %s end %s", val_label_feature['day'].min(),
             val_label_feature['day'].max(), test['day'].min(), test['day'].max())

[2019-12-09 08:40:34,345] INFO in <ipython-input-8-5c7aac178bca>: train_label_feature (6895493, 5)
[2019-12-09 08:40:34,901] INFO in <ipython-input-8-5c7aac178bca>: val_label_feature (7583553, 5)
[2019-12-09 08:40:35,146] INFO in <ipython-input-8-5c7aac178bca>: train feature start 3838 end 3860, label start 3861 end 3867
[2019-12-09 08:40:35,173] INFO in <ipython-input-8-5c7aac178bca>: test feature start 3845 end 3867, label start 3868 end 3874


```python
print(train_label_feature_start, train_label_feature_end)
print(train_ans_feature_start, train_ans_feature_end)
print(val_label_feature_start, val_label_feature_end)
print(val_ans_feature_start, val_ans_feature_end)
```
```text
3838 3860
3810 3860
3845 3867
3817 3867
```

确定ans的时间范围。

In [9]:
train_ans_feature = ans[(ans['a_day'] >= train_ans_feature_start) & (ans['a_day'] <= train_ans_feature_end)]

val_ans_feature = ans[(ans['a_day'] >= val_ans_feature_start) & (ans['a_day'] <= val_ans_feature_end)]

logging.info("train ans feature %s, start %s end %s", train_ans_feature.shape, train_ans_feature['a_day'].min(),
             train_ans_feature['a_day'].max())

logging.info("val ans feature %s, start %s end %s", val_ans_feature.shape, val_ans_feature['a_day'].min(),
             val_ans_feature['a_day'].max())

fea_cols = ['is_good', 'is_rec', 'is_dest', 'has_img', 'has_video', 'word_count',
            'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
            'reci_xxx', 'reci_no_help', 'reci_dis', 'diff_qa_days']

[2019-12-09 08:40:38,163] INFO in <ipython-input-9-502ad99fdbbb>: train ans feature (3700178, 23), start 3810 end 3860
[2019-12-09 08:40:38,173] INFO in <ipython-input-9-502ad99fdbbb>: val ans feature (3992334, 23), start 3817 end 3867


## 2.5 特征提取

In [10]:
def extract_feature1(target, label_feature, ans_feature):
    # 问题特征
    t1 = label_feature.groupby('qid')['label'].agg(['mean', 'sum', 'std', 'count', 
                                                    'max', 'min']).reset_index()
    t1.columns = ['qid', 'q_inv_mean', 'q_inv_sum', 'q_inv_std', 'q_inv_count', 
                  'q_inv_max', 'q_inv_min']
    target = pd.merge(target, t1, on='qid', how='left')

    # 用户特征
    t1 = label_feature.groupby('uid')['label'].agg(['mean', 'sum', 'std', 'count', 
                                                    'max', 'min']).reset_index()
    t1.columns = ['uid', 'u_inv_mean', 'u_inv_sum', 'u_inv_std', 'u_inv_count', 
                  'u_inv_max', 'u_inv_min']
    target = pd.merge(target, t1, on='uid', how='left')
    #
    # train_size = len(train)
    # data = pd.concat((train, test), sort=True)

    # 回答部分特征

    t1 = ans_feature.groupby('qid')['aid'].count().reset_index()
    t1.columns = ['qid', 'q_ans_count']
    target = pd.merge(target, t1, on='qid', how='left')

    t1 = ans_feature.groupby('uid')['aid'].count().reset_index()
    t1.columns = ['uid', 'u_ans_count']
    target = pd.merge(target, t1, on='uid', how='left')

    for col in fea_cols:
        t1 = ans_feature.groupby('uid')[col].agg(['sum', 'max', 'mean', 
                                                  'std', 'count', 'min']).reset_index()
        t1.columns = ['uid', f'u_{col}_sum', f'u_{col}_max', f'u_{col}_mean',
                      f'u_{col}_std', f'u_{col}_count', f'u_{col}_min']
        target = pd.merge(target, t1, on='uid', how='left')

        t1 = ans_feature.groupby('qid')[col].agg(['sum', 'max', 'mean',
                                                  'std', 'count', 'min']).reset_index()
        t1.columns = ['qid', f'q_{col}_sum', f'q_{col}_max', f'q_{col}_mean',
                      f'q_{col}_std', f'q_{col}_count', f'q_{col}_min']
        target = pd.merge(target, t1, on='qid', how='left')
        logging.info("extract %s", col)
    return target


train_label = extract_feature1(train_label, train_label_feature, train_ans_feature)
test = extract_feature1(test, val_label_feature, val_ans_feature)

[2019-12-09 08:41:15,101] INFO in <ipython-input-10-0304f9fa763e>: extract is_good
[2019-12-09 08:41:27,275] INFO in <ipython-input-10-0304f9fa763e>: extract is_rec
[2019-12-09 08:41:39,747] INFO in <ipython-input-10-0304f9fa763e>: extract is_dest
[2019-12-09 08:41:52,680] INFO in <ipython-input-10-0304f9fa763e>: extract has_img
[2019-12-09 08:42:06,051] INFO in <ipython-input-10-0304f9fa763e>: extract has_video
[2019-12-09 08:42:19,747] INFO in <ipython-input-10-0304f9fa763e>: extract word_count
[2019-12-09 08:42:33,787] INFO in <ipython-input-10-0304f9fa763e>: extract reci_cheer
[2019-12-09 08:42:48,374] INFO in <ipython-input-10-0304f9fa763e>: extract reci_uncheer
[2019-12-09 08:43:03,311] INFO in <ipython-input-10-0304f9fa763e>: extract reci_comment
[2019-12-09 08:43:18,470] INFO in <ipython-input-10-0304f9fa763e>: extract reci_mark
[2019-12-09 08:43:33,977] INFO in <ipython-input-10-0304f9fa763e>: extract reci_tks
[2019-12-09 08:43:50,169] INFO in <ipython-input-10-0304f9fa763e>: 

特征提取结束。

In [11]:
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)
assert len(test) == sub_size

[2019-12-09 08:47:53,725] INFO in <ipython-input-11-6bad7a600272>: train shape (2593669, 199), test shape (1141683, 198)


## 2.6 加载用户

In [12]:
user = pd.read_csv(f'{base_path}/member_info_0926.txt', header=None, sep='\t')
user.columns = ['uid', 'gender', 'creat_keyword', 'level', 'hot', 'reg_type', 'reg_plat', 'freq', 'uf_b1', 'uf_b2',
                'uf_b3', 'uf_b4', 'uf_b5', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5', 'score', 'follow_topic',
                'inter_topic']
del user['follow_topic'], user['inter_topic']
logging.info("user %s", user.shape)

unq = user.nunique()
logging.info("user unq %s", unq)

for x in unq[unq == 1].index:
    del user[x]
    logging.info('del unq==1 %s', x)

t = user.dtypes
cats = [x for x in t[t == 'object'].index if x not in ['follow_topic', 'inter_topic', 'uid']]
logging.info("user cat %s", cats)

for d in cats:
    lb = LabelEncoder()
    user[d] = lb.fit_transform(user[d])
    logging.info('encode %s', d)

q_lb = LabelEncoder()
q_lb.fit(list(train_label['qid'].astype(str).values) + list(test['qid'].astype(str).values))
train_label['qid_enc'] = q_lb.transform(train_label['qid'])
test['qid_enc'] = q_lb.transform(test['qid'])

u_lb = LabelEncoder()
u_lb.fit(user['uid'])
train_label['uid_enc'] = u_lb.transform(train_label['uid'])
test['uid_enc'] = u_lb.transform(test['uid'])

[2019-12-09 08:48:02,122] INFO in <ipython-input-12-f4f8e8f0ac9b>: user (1931654, 19)
[2019-12-09 08:48:05,021] INFO in <ipython-input-12-f4f8e8f0ac9b>: user unq uid              1931654
gender                 3
creat_keyword          1
level                  1
hot                    1
reg_type               1
reg_plat               1
freq                   5
uf_b1                  2
uf_b2                  2
uf_b3                  2
uf_b4                  2
uf_b5                  2
uf_c1               2561
uf_c2                291
uf_c3                428
uf_c4               1556
uf_c5                  2
score                732
dtype: int64
[2019-12-09 08:48:05,047] INFO in <ipython-input-12-f4f8e8f0ac9b>: del unq==1 creat_keyword
[2019-12-09 08:48:05,053] INFO in <ipython-input-12-f4f8e8f0ac9b>: del unq==1 level
[2019-12-09 08:48:05,054] INFO in <ipython-input-12-f4f8e8f0ac9b>: del unq==1 hot
[2019-12-09 08:48:05,183] INFO in <ipython-input-12-f4f8e8f0ac9b>: del unq==1 reg_type
[2019

merge user

In [13]:
train_label = pd.merge(train_label, user, on='uid', how='left')
test = pd.merge(test, user, on='uid', how='left')
logging.info("train shape %s, test shape %s", train_label.shape, test.shape)

[2019-12-09 08:48:40,127] INFO in <ipython-input-13-a0af6fdcbd8e>: train shape (2593669, 214), test shape (1141683, 213)


及时清理内存。

In [14]:
del ans, train_label_feature, train_ans_feature, val_label_feature, val_ans_feature, user

In [15]:
data = pd.concat((train_label, test), axis=0, sort=True)
len_train_label = len(train_label)

# 暂存清理变量
save_param(train_label, 'train_label')
del train_label
del test

count编码。

In [17]:
data

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_min,...,uid_enc,uid_enc_count,qid_enc_count,gender_count,freq_count,uf_c1_count,uf_c2_count,uf_c3_count,uf_c4_count,uf_c5_count
0,3865,5,3,22,0.0,NaN,NaN,NaN,NaN,NaN,...,1508099,0.000004,0.000369,1.000000,0.961323,0.022204,0.371510,0.217985,0.062753,1.0
1,3862,5,3,15,0.0,NaN,NaN,NaN,NaN,NaN,...,657986,0.000004,0.000036,1.000000,0.961323,0.024503,0.111695,0.443770,0.002973,1.0
2,3867,2,3,4,0.0,32.0,32.0,13.0,9.531250,0.0,...,1272354,0.000022,0.001341,1.000000,0.291847,0.006308,0.034012,0.400961,0.044143,1.0
3,3861,1,2,20,0.0,3.0,3.0,234.0,222.333333,215.0,...,909155,0.000004,0.000018,0.000000,1.000000,0.024503,0.111695,0.443770,0.002973,1.0
4,3862,5,1,8,1.0,NaN,NaN,NaN,NaN,NaN,...,1856020,0.000065,0.000027,0.009572,0.961323,0.005711,0.007464,0.438832,0.126329,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141678,3869,5,3,20,NaN,NaN,NaN,NaN,NaN,NaN,...,504329,0.000011,0.000090,1.000000,0.961323,0.050396,0.371510,0.045014,1.000000,1.0
1141679,3872,2,3,21,NaN,NaN,NaN,NaN,NaN,NaN,...,1066118,0.000004,0.000009,1.000000,0.291847,1.000000,1.000000,0.214137,1.000000,1.0
1141680,3871,1,3,15,NaN,NaN,NaN,NaN,NaN,NaN,...,435861,0.000011,0.000045,1.000000,1.000000,0.000098,0.002885,0.130101,1.000000,1.0
1141681,3871,5,3,8,NaN,NaN,NaN,NaN,NaN,NaN,...,1287651,0.000014,0.000252,1.000000,0.961323,1.000000,1.000000,1.000000,0.069500,1.0


In [16]:
count_fea = ['uid_enc', 'qid_enc', 'gender', 'freq', 'uf_c1', 'uf_c2', 'uf_c3', 'uf_c4', 'uf_c5']
with tqdm(total=len(count_fea)) as pbar:
    for feat in count_fea:
        col_name = '{}_count'.format(feat)
        data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
        data.loc[data[col_name] < 2, feat] = -1
        data[feat] += 1
        data[col_name] = data[feat].map(data[feat].value_counts().astype(int))
        data[col_name] = (data[col_name] - data[col_name].min()) / (data[col_name].max() - data[col_name].min())
        pbar.update(1)

100%|██████████| 9/9 [01:27<00:00,  9.76s/it]


## 2.7 处理数据

压缩数据。

In [18]:
t = data.dtypes
for x in t[t == 'int64'].index:
    data[x] = data[x].astype('int32')

for x in t[t == 'float64'].index:
    data[x] = data[x].astype('float32')

data['wk'] = data['day'] % 7

feature_cols = [x for x in data.columns if x not in ('label', 'uid', 'qid', 'dt', 'day')]

target编码。

In [19]:
logging.info("feature size %s", len(feature_cols))

# 恢复数据
train_label = read_param('train_label')
    
X_train_all = data.iloc[:len_train_label][feature_cols]
y_train_all = data.iloc[:len_train_label]['label']

test = data.iloc[len_train_label:]
# with open(f'{base_path}/temp/test.txt', 'wb') as f:
#     pickle.dump(test, f)
#     del test
    
assert len(data) - len_train_label == sub_size

[2019-12-09 08:57:10,010] INFO in <ipython-input-19-5a54c180d6dd>: feature size 220


# 3 训练树

## 3.1 预处理数据

In [20]:
logging.info("train shape %s, test shape %s", 
             data.iloc[:len_train_label].shape, 
             data.iloc[len_train_label:].shape)

save_param(data, 'data')
del data


# 分层采样，交叉验证（切分5片）
fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# 但只用了其中一次切片
for index, (train_idx, val_idx) in enumerate(fold.split(X=X_train_all, y=y_train_all)):
    break

X_train, X_val, y_train, y_val = X_train_all.iloc[train_idx][feature_cols], X_train_all.iloc[val_idx][feature_cols], \
                                 y_train_all.iloc[train_idx], \
                                 y_train_all.iloc[val_idx]
# del X_train_all

[2019-12-09 08:57:18,247] INFO in <ipython-input-20-9e41c446c44b>: train shape (2593669, 224), test shape (1141683, 224)


## 3.2 存取数据

存变量

In [21]:
save_param(X_train, 'X_train')
save_param(y_train, 'y_train')
save_param(X_val, 'X_val')
save_param(y_val, 'y_val')
save_param(test, 'test')
save_param(sub, 'sub')
save_param(X_train_all, 'X_train_all')
save_param(y_train_all, 'y_train_all')
save_param(feature_cols, 'feature_cols')

取变量

In [4]:
# X_train = read_param('X_train')
# y_train = read_param('y_train')
X_val = read_param('X_val')
y_val = read_param('y_val')
test = read_param('test')
sub = read_param('sub')
X_train_all = read_param('X_train_all')
y_train_all = read_param('y_train_all')
feature_cols = read_param('feature_cols')

## 3.3 LGBM分类器训练

### 3.3.1 LGBMClassifier

On ('dart', lr=0.5, n_estimators=200)
* Epoch 1: TPE using 33/33 trials with best loss 0.162092
* Epoch 2: TPE using 10/10 trials with best loss 0.162055
* Epoch 3: TPE using 8/8 trials with best loss 0.162175
* Epoch 4: TPE using 23/23 trials with best loss 0.161951
* Epoch 5: TPE using 15/15 trials with best loss 0.161962
* Epoch 6: TPE using 45/45 trials with best loss 0.161878

|param_name|epoch 1|epoch 2|epoch 3|epoch 4|epoch 5|final|
|:--|:--|:--|:--|:--|:--|:--|
|max_depth|20|24|22|20|20|20|
|num_leaves|200|200|180|180|190|190|
|min_child_samples|220|160|220|210|210|210|
|min_child_weight|0.056|0.045|0.045|0.055|0.055|0.055|
|feature_fraction|0.7|0.7|0.65|0.6|0.6|0.7|
|reg_alpha|0.001|0.029|0.001|0.009|0.005|0.005|
|reg_lambda|13.5|15.5|15|16|18|21|

In [5]:
model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.005,
                           n_estimators=10000, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           feature_fraction=0.7,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
                           seed=666, 
                           silent=True,
                           device='gpu',)
model_lgb.fit(X_train_all, y_train_all,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)

[1]	valid_0's auc: 0.787704	valid_0's binary_logloss: 0.438868
[2]	valid_0's auc: 0.79909	valid_0's binary_logloss: 0.438019
[3]	valid_0's auc: 0.799961	valid_0's binary_logloss: 0.437371
[4]	valid_0's auc: 0.803661	valid_0's binary_logloss: 0.436557
[5]	valid_0's auc: 0.806345	valid_0's binary_logloss: 0.435976
[6]	valid_0's auc: 0.808438	valid_0's binary_logloss: 0.435875
[7]	valid_0's auc: 0.809585	valid_0's binary_logloss: 0.43503
[8]	valid_0's auc: 0.809034	valid_0's binary_logloss: 0.434459
[9]	valid_0's auc: 0.809343	valid_0's binary_logloss: 0.434513
[10]	valid_0's auc: 0.809501	valid_0's binary_logloss: 0.434262
[11]	valid_0's auc: 0.809726	valid_0's binary_logloss: 0.433481
[12]	valid_0's auc: 0.810968	valid_0's binary_logloss: 0.432876
[13]	valid_0's auc: 0.81099	valid_0's binary_logloss: 0.432888
[14]	valid_0's auc: 0.811472	valid_0's binary_logloss: 0.432103
[15]	valid_0's auc: 0.811664	valid_0's binary_logloss: 0.431584
[16]	valid_0's auc: 0.811793	valid_0's binary_loglos

[130]	valid_0's auc: 0.814706	valid_0's binary_logloss: 0.444897
[131]	valid_0's auc: 0.814718	valid_0's binary_logloss: 0.445289
[132]	valid_0's auc: 0.814788	valid_0's binary_logloss: 0.44469
[133]	valid_0's auc: 0.81487	valid_0's binary_logloss: 0.44401
[134]	valid_0's auc: 0.814862	valid_0's binary_logloss: 0.443385
[135]	valid_0's auc: 0.814866	valid_0's binary_logloss: 0.443756
[136]	valid_0's auc: 0.814827	valid_0's binary_logloss: 0.443147
[137]	valid_0's auc: 0.814825	valid_0's binary_logloss: 0.458112
[138]	valid_0's auc: 0.814881	valid_0's binary_logloss: 0.457332
[139]	valid_0's auc: 0.814891	valid_0's binary_logloss: 0.457615
[140]	valid_0's auc: 0.814898	valid_0's binary_logloss: 0.457998
[141]	valid_0's auc: 0.814937	valid_0's binary_logloss: 0.45735
[142]	valid_0's auc: 0.815058	valid_0's binary_logloss: 0.456601
[143]	valid_0's auc: 0.815048	valid_0's binary_logloss: 0.456988
[144]	valid_0's auc: 0.815032	valid_0's binary_logloss: 0.456347
[145]	valid_0's auc: 0.815028

[257]	valid_0's auc: 0.815868	valid_0's binary_logloss: 0.449417
[258]	valid_0's auc: 0.815869	valid_0's binary_logloss: 0.449745
[259]	valid_0's auc: 0.815886	valid_0's binary_logloss: 0.449145
[260]	valid_0's auc: 0.815924	valid_0's binary_logloss: 0.448434
[261]	valid_0's auc: 0.815929	valid_0's binary_logloss: 0.44881
[262]	valid_0's auc: 0.815932	valid_0's binary_logloss: 0.448106
[263]	valid_0's auc: 0.815938	valid_0's binary_logloss: 0.448485
[264]	valid_0's auc: 0.81594	valid_0's binary_logloss: 0.448788
[265]	valid_0's auc: 0.815952	valid_0's binary_logloss: 0.448071
[266]	valid_0's auc: 0.815946	valid_0's binary_logloss: 0.448435
[267]	valid_0's auc: 0.815899	valid_0's binary_logloss: 0.44777
[268]	valid_0's auc: 0.815893	valid_0's binary_logloss: 0.448166
[269]	valid_0's auc: 0.815882	valid_0's binary_logloss: 0.448535
[270]	valid_0's auc: 0.815872	valid_0's binary_logloss: 0.448841
[271]	valid_0's auc: 0.815891	valid_0's binary_logloss: 0.44814
[272]	valid_0's auc: 0.815895

[384]	valid_0's auc: 0.816971	valid_0's binary_logloss: 0.445916
[385]	valid_0's auc: 0.816973	valid_0's binary_logloss: 0.447908
[386]	valid_0's auc: 0.81699	valid_0's binary_logloss: 0.447191
[387]	valid_0's auc: 0.817002	valid_0's binary_logloss: 0.44655
[388]	valid_0's auc: 0.817045	valid_0's binary_logloss: 0.445905
[389]	valid_0's auc: 0.817045	valid_0's binary_logloss: 0.446261
[390]	valid_0's auc: 0.817035	valid_0's binary_logloss: 0.446648
[391]	valid_0's auc: 0.817046	valid_0's binary_logloss: 0.44594
[392]	valid_0's auc: 0.817062	valid_0's binary_logloss: 0.445279
[393]	valid_0's auc: 0.817107	valid_0's binary_logloss: 0.444643
[394]	valid_0's auc: 0.817107	valid_0's binary_logloss: 0.444986
[395]	valid_0's auc: 0.817101	valid_0's binary_logloss: 0.44533
[396]	valid_0's auc: 0.817148	valid_0's binary_logloss: 0.444681
[397]	valid_0's auc: 0.817139	valid_0's binary_logloss: 0.444085
[398]	valid_0's auc: 0.817135	valid_0's binary_logloss: 0.444449
[399]	valid_0's auc: 0.817087

[511]	valid_0's auc: 0.817615	valid_0's binary_logloss: 0.431416
[512]	valid_0's auc: 0.817611	valid_0's binary_logloss: 0.431739
[513]	valid_0's auc: 0.817589	valid_0's binary_logloss: 0.43117
[514]	valid_0's auc: 0.817571	valid_0's binary_logloss: 0.430606
[515]	valid_0's auc: 0.81757	valid_0's binary_logloss: 0.430946
[516]	valid_0's auc: 0.817608	valid_0's binary_logloss: 0.430387
[517]	valid_0's auc: 0.817606	valid_0's binary_logloss: 0.430719
[518]	valid_0's auc: 0.817663	valid_0's binary_logloss: 0.430191
[519]	valid_0's auc: 0.817646	valid_0's binary_logloss: 0.429634
[520]	valid_0's auc: 0.817701	valid_0's binary_logloss: 0.429041
[521]	valid_0's auc: 0.81777	valid_0's binary_logloss: 0.428461
[522]	valid_0's auc: 0.817768	valid_0's binary_logloss: 0.42878
[523]	valid_0's auc: 0.81778	valid_0's binary_logloss: 0.428204
[524]	valid_0's auc: 0.817767	valid_0's binary_logloss: 0.427676
[525]	valid_0's auc: 0.817791	valid_0's binary_logloss: 0.4271
[526]	valid_0's auc: 0.817788	va

...

[9519]	valid_0's auc: 0.849892	valid_0's binary_logloss: 0.320182
[9520]	valid_0's auc: 0.849896	valid_0's binary_logloss: 0.320177
[9521]	valid_0's auc: 0.849898	valid_0's binary_logloss: 0.32018
[9522]	valid_0's auc: 0.849902	valid_0's binary_logloss: 0.320174
[9523]	valid_0's auc: 0.849904	valid_0's binary_logloss: 0.320168
[9524]	valid_0's auc: 0.849907	valid_0's binary_logloss: 0.320162
[9525]	valid_0's auc: 0.849908	valid_0's binary_logloss: 0.320165
[9526]	valid_0's auc: 0.849911	valid_0's binary_logloss: 0.32016
[9527]	valid_0's auc: 0.849913	valid_0's binary_logloss: 0.320154
[9528]	valid_0's auc: 0.849916	valid_0's binary_logloss: 0.320148
[9529]	valid_0's auc: 0.849917	valid_0's binary_logloss: 0.320151
[9530]	valid_0's auc: 0.849919	valid_0's binary_logloss: 0.320146
[9531]	valid_0's auc: 0.849921	valid_0's binary_logloss: 0.320149
[9532]	valid_0's auc: 0.849923	valid_0's binary_logloss: 0.320144
[9533]	valid_0's auc: 0.849925	valid_0's binary_logloss: 0.320138
[9534]	valid

[9644]	valid_0's auc: 0.85012	valid_0's binary_logloss: 0.319994
[9645]	valid_0's auc: 0.850121	valid_0's binary_logloss: 0.319996
[9646]	valid_0's auc: 0.850122	valid_0's binary_logloss: 0.319998
[9647]	valid_0's auc: 0.850123	valid_0's binary_logloss: 0.320004
[9648]	valid_0's auc: 0.850125	valid_0's binary_logloss: 0.320007
[9649]	valid_0's auc: 0.850126	valid_0's binary_logloss: 0.32001
[9650]	valid_0's auc: 0.850129	valid_0's binary_logloss: 0.320004
[9651]	valid_0's auc: 0.85013	valid_0's binary_logloss: 0.320006
[9652]	valid_0's auc: 0.850131	valid_0's binary_logloss: 0.320008
[9653]	valid_0's auc: 0.850132	valid_0's binary_logloss: 0.32001
[9654]	valid_0's auc: 0.850133	valid_0's binary_logloss: 0.320012
[9655]	valid_0's auc: 0.850135	valid_0's binary_logloss: 0.320006
[9656]	valid_0's auc: 0.850136	valid_0's binary_logloss: 0.320008
[9657]	valid_0's auc: 0.850137	valid_0's binary_logloss: 0.32001
[9658]	valid_0's auc: 0.850139	valid_0's binary_logloss: 0.320004
[9659]	valid_0'

[9769]	valid_0's auc: 0.850335	valid_0's binary_logloss: 0.319842
[9770]	valid_0's auc: 0.850336	valid_0's binary_logloss: 0.319845
[9771]	valid_0's auc: 0.850337	valid_0's binary_logloss: 0.319847
[9772]	valid_0's auc: 0.850338	valid_0's binary_logloss: 0.31985
[9773]	valid_0's auc: 0.85034	valid_0's binary_logloss: 0.319853
[9774]	valid_0's auc: 0.850344	valid_0's binary_logloss: 0.319847
[9775]	valid_0's auc: 0.850346	valid_0's binary_logloss: 0.319849
[9776]	valid_0's auc: 0.850346	valid_0's binary_logloss: 0.319844
[9777]	valid_0's auc: 0.85035	valid_0's binary_logloss: 0.319837
[9778]	valid_0's auc: 0.850352	valid_0's binary_logloss: 0.319841
[9779]	valid_0's auc: 0.850356	valid_0's binary_logloss: 0.319836
[9780]	valid_0's auc: 0.850357	valid_0's binary_logloss: 0.319839
[9781]	valid_0's auc: 0.850358	valid_0's binary_logloss: 0.319842
[9782]	valid_0's auc: 0.85036	valid_0's binary_logloss: 0.319836
[9783]	valid_0's auc: 0.850361	valid_0's binary_logloss: 0.319839
[9784]	valid_0

[9894]	valid_0's auc: 0.850546	valid_0's binary_logloss: 0.319648
[9895]	valid_0's auc: 0.850547	valid_0's binary_logloss: 0.319652
[9896]	valid_0's auc: 0.850549	valid_0's binary_logloss: 0.319645
[9897]	valid_0's auc: 0.850551	valid_0's binary_logloss: 0.31965
[9898]	valid_0's auc: 0.850553	valid_0's binary_logloss: 0.319645
[9899]	valid_0's auc: 0.850555	valid_0's binary_logloss: 0.319638
[9900]	valid_0's auc: 0.850556	valid_0's binary_logloss: 0.319641
[9901]	valid_0's auc: 0.850557	valid_0's binary_logloss: 0.319643
[9902]	valid_0's auc: 0.850559	valid_0's binary_logloss: 0.319645
[9903]	valid_0's auc: 0.85056	valid_0's binary_logloss: 0.319649
[9904]	valid_0's auc: 0.850564	valid_0's binary_logloss: 0.319643
[9905]	valid_0's auc: 0.850565	valid_0's binary_logloss: 0.319645
[9906]	valid_0's auc: 0.850567	valid_0's binary_logloss: 0.319639
[9907]	valid_0's auc: 0.850568	valid_0's binary_logloss: 0.319641
[9908]	valid_0's auc: 0.850569	valid_0's binary_logloss: 0.319644
[9909]	valid

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
               device='gpu', feature_fraction=0.7, importance_type='split',
               learning_rate=0.005, max_depth=20, min_child_samples=210,
               min_child_weight=0.055, min_split_gain=0.0, n_estimators=10000,
               n_jobs=16, num_leaves=190, objective='binary', random_state=None,
               reg_alpha=0.005, reg_lambda=21, seed=666, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [6]:
save_param(model_lgb, 'model_lgb')

### 3.3.2 lgb.cv调参

In [7]:
# train_data = lgb.Dataset(X_train_all[int(0.3*len(X_train_all)):], y_train_all[int(0.3*len(X_train_all)):])

model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.5,
                           n_estimators=200, 
                           max_depth=28,
                           num_leaves=1100,
                           min_child_samples=21,
                           min_child_weight=0.001,
                           feature_fraction = 0.7,
                           bagging_fraction = 1,
                           bagging_freq = 2,
                           reg_alpha = 0.001,
                           reg_lambda = 8,
                           cat_smooth = 0,
                           n_jobs=16,
                           silent=True,
                           device='gpu',)

cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                nfold=3, metrics=['logloss', 'auc'], verbose_eval=50,
                early_stopping_rounds=10, stratified=False, shuffle=True,)

print('best n_estimators:', len(cv_res['auc-mean']))
print('best cv score:', cv_res['auc-mean'][-1])

[50]	cv_agg's auc: 0.830227 + 0.00038161
[100]	cv_agg's auc: 0.833206 + 0.000225613
[150]	cv_agg's auc: 0.833865 + 0.000152819
[200]	cv_agg's auc: 0.833253 + 0.000188247


NameError: name 'cv_results' is not defined

学习率为 0.5 时，最优迭代次数 25 。
学习率为 0.2 时，最优迭代次数 276 。代入 GridSearchCV 调参。

#### 3.3.2.1 max_depth, num_leaves

最佳设定值为： {'max_depth': 6, 'num_leaves': 19}。

其正确率为： 0.8526213960944669

In [47]:
from sklearn.model_selection import GridSearchCV
model_lgb = LGBMClassifier(learning_rate=0.2, n_estimators=276, 
                           n_jobs=-1, objective='binary', 
                           seed=1000, silent=True)
params_test1 = {
    'max_depth': [7, 10, 13],
    'num_leaves': [20, 50, 100],
}
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test1, cv=5, 
                        verbose=1, n_jobs=8)

gsearch1.fit(X_train_all[:int(0.1*(len(X_train_all)))], 
             y_train_all[:int(0.1*(len(X_train_all)))])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  45 out of  45 | elapsed:  3.2min finished


({'mean_fit_time': array([22.64980655, 30.14839115, 34.6697649 , 23.43322554, 30.88116717,
         42.11346478, 23.42752047, 31.42543778, 35.29486856]),
  'std_fit_time': array([0.77179366, 1.31676194, 1.32452173, 0.65123278, 0.4495243 ,
         2.71318179, 0.40253905, 1.12696808, 2.57828579]),
  'mean_score_time': array([0.54789038, 0.70412478, 0.7134861 , 0.60295467, 0.63511691,
         0.78498187, 0.54346294, 0.61726995, 0.55713482]),
  'std_score_time': array([0.01985728, 0.03341938, 0.04992322, 0.05496516, 0.06476402,
         0.05987709, 0.0584149 , 0.03901728, 0.05014078]),
  'param_max_depth': masked_array(data=[7, 7, 7, 10, 10, 10, 13, 13, 13],
               mask=[False, False, False, False, False, False, False, False,
                     False],
         fill_value='?',
              dtype=object),
  'param_num_leaves': masked_array(data=[20, 50, 100, 20, 50, 100, 20, 50, 100],
               mask=[False, False, False, False, False, False, False, False,
                 

#### 3.3.2.2 min_child_samples, min_child_weight

 {'min_child_samples': 20, 'min_child_weight': 0.001}
 
 0.8526213960944669

In [33]:
from sklearn.model_selection import GridSearchCV
model_lgb = LGBMClassifier(learning_rate=0.5, n_estimators=25, 
                           num_leaves=19, max_depth=6,
                           n_jobs=-1, objective='binary', 
                           seed=1000, silent=True)
params_test2 = {
    'min_child_samples': [18, 19, 20],
    'min_child_weight':[0.001]
}
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test2, cv=5, 
                        verbose=1, n_jobs=8)

gsearch1.fit(X_train_all[:int(0.1*(len(X_train_all)))], 
             y_train_all[:int(0.1*(len(X_train_all)))])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  15 out of  15 | elapsed:   13.5s finished


({'mean_fit_time': array([6.0507174 , 5.63760686, 5.50885849]),
  'std_fit_time': array([0.14310271, 0.0971775 , 0.05686421]),
  'mean_score_time': array([0.34664764, 0.33750153, 0.30462813]),
  'std_score_time': array([0.04410642, 0.01698014, 0.00578745]),
  'param_min_child_samples': masked_array(data=[18, 19, 20],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'param_min_child_weight': masked_array(data=[0.001, 0.001, 0.001],
               mask=[False, False, False],
         fill_value='?',
              dtype=object),
  'params': [{'min_child_samples': 18, 'min_child_weight': 0.001},
   {'min_child_samples': 19, 'min_child_weight': 0.001},
   {'min_child_samples': 20, 'min_child_weight': 0.001}],
  'split0_test_score': array([0.85071519, 0.85098508, 0.85160196]),
  'split1_test_score': array([0.85321844, 0.85262082, 0.85235093]),
  'split2_test_score': array([0.85231238, 0.8519461 , 0.85198465]),
  'split3_test_score': array([0.

#### 3.3.2.3 subsample, subsample_freq

发现默认值最好。

In [37]:
from sklearn.model_selection import GridSearchCV
model_lgb = LGBMClassifier(learning_rate=0.5, n_estimators=25, 
                           num_leaves=19, max_depth=6,
                           min_child_samples=20, min_child_weight=0.001,
                           n_jobs=-1, objective='binary', 
                           seed=1000, silent=True)
params_test3 = {
    'subsample': [0.1, 1],
    'subsample_freq':[0]
}
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test3, cv=5, 
                        verbose=1, n_jobs=8)

gsearch1.fit(X_train_all[:int(0.1*(len(X_train_all)))], 
             y_train_all[:int(0.1*(len(X_train_all)))])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  10 out of  10 | elapsed:    9.5s finished


({'mean_fit_time': array([7.57890711, 7.3646234 ]),
  'std_fit_time': array([0.19705638, 0.4705335 ]),
  'mean_score_time': array([0.36439962, 0.37709317]),
  'std_score_time': array([0.02826785, 0.0433045 ]),
  'param_subsample': masked_array(data=[0.1, 1],
               mask=[False, False],
         fill_value='?',
              dtype=object),
  'param_subsample_freq': masked_array(data=[0, 0],
               mask=[False, False],
         fill_value='?',
              dtype=object),
  'params': [{'subsample': 0.1, 'subsample_freq': 0},
   {'subsample': 1, 'subsample_freq': 0}],
  'split0_test_score': array([0.85160196, 0.85160196]),
  'split1_test_score': array([0.85235093, 0.85235093]),
  'split2_test_score': array([0.85198465, 0.85198465]),
  'split3_test_score': array([0.85277505, 0.85277505]),
  'split4_test_score': array([0.85439439, 0.85439439]),
  'mean_test_score': array([0.8526214, 0.8526214]),
  'std_test_score': array([0.00096798, 0.00096798]),
  'rank_test_score': array(

#### 3.3.2.4 reg_alpha, reg_lambda

使用默认值效果最好。

In [38]:
from sklearn.model_selection import GridSearchCV
model_lgb = LGBMClassifier(learning_rate=0.5, n_estimators=25, 
                           num_leaves=19, max_depth=6,
                           min_child_samples=20, min_child_weight=0.001,
                           n_jobs=-1, objective='binary', 
                           seed=1000, silent=True)
params_test4 = {
    'reg_alpha': [0, 0.001, 0.1, 0.5],
    'reg_lambda':[0, 0.001, 0.1, 0.5]
}
gsearch1 = GridSearchCV(estimator=model_lgb, param_grid=params_test4, cv=5, 
                        verbose=1, n_jobs=8)

gsearch1.fit(X_train_all[:int(0.1*(len(X_train_all)))], 
             y_train_all[:int(0.1*(len(X_train_all)))])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:   24.5s
[Parallel(n_jobs=12)]: Done  80 out of  80 | elapsed:   56.2s finished


({'mean_fit_time': array([7.71533136, 7.9726099 , 7.3448319 , 7.52182326, 7.39398718,
         7.21040425, 7.53177915, 7.43859038, 7.13111324, 7.4581872 ,
         7.32337446, 7.13512297, 7.56366177, 7.24008512, 7.06009641,
         6.18112383]),
  'std_fit_time': array([0.35095547, 0.33313138, 0.3301298 , 0.32133429, 0.23104272,
         0.288994  , 0.21607216, 0.25020613, 0.22608667, 0.06207035,
         0.23886111, 0.06106141, 0.21793575, 0.16834756, 0.25553736,
         0.57317338]),
  'mean_score_time': array([0.40201359, 0.36251307, 0.44310536, 0.39268341, 0.36745706,
         0.39516087, 0.36227026, 0.38925633, 0.39509387, 0.33925614,
         0.37908244, 0.32322288, 0.37539716, 0.36074023, 0.31853189,
         0.31228409]),
  'std_score_time': array([0.02233388, 0.02707548, 0.06015347, 0.07452821, 0.04593065,
         0.03961645, 0.03691229, 0.04516038, 0.06953506, 0.02784691,
         0.04698842, 0.0129001 , 0.03101683, 0.03339664, 0.01815995,
         0.0235051 ]),
  'param_r

#### 3.3.2.5 评估

In [41]:
import lightgbm as lgb
data_train = lgb.Dataset(X_train_all, y_train_all)

In [42]:
params = {
    'boosting_type': 'gbdt', 
    'objective': 'binary', 

    'learning_rate': 0.005, 
    'max_depth': 6,
    'num_leaves': 19, 
    'min_child_samples': 20,
    'min_child_weight': 0.001,

    'subsample': 1, 
    'colsample_bytree': 0.7, 
}
cv_results = lgb.cv(params, data_train,
                    num_boost_round=10000, nfold=5, stratified=False, shuffle=True,
                    metrics=['logloss', 'auc'], early_stopping_rounds=50, 
                    verbose_eval=50, show_stdv=True, seed=0)

print('best n_estimators:', len(cv_results['auc-mean']))
print('best cv score:', cv_results['auc-mean'][-1])

[50]	cv_agg's auc: 0.782697 + 0.000866683
[100]	cv_agg's auc: 0.786328 + 0.000828643
[150]	cv_agg's auc: 0.79002 + 0.000743684
[200]	cv_agg's auc: 0.793664 + 0.0007177
[250]	cv_agg's auc: 0.796969 + 0.000706312
[300]	cv_agg's auc: 0.799941 + 0.000723938
[350]	cv_agg's auc: 0.802602 + 0.000717428
[400]	cv_agg's auc: 0.80476 + 0.000745901
[450]	cv_agg's auc: 0.806776 + 0.000749207
[500]	cv_agg's auc: 0.808439 + 0.000758047
[550]	cv_agg's auc: 0.809801 + 0.000738387
[600]	cv_agg's auc: 0.811044 + 0.000743624
[650]	cv_agg's auc: 0.812169 + 0.00073528
[700]	cv_agg's auc: 0.813189 + 0.000727163
[750]	cv_agg's auc: 0.814024 + 0.000717278
[800]	cv_agg's auc: 0.814807 + 0.000712655
[850]	cv_agg's auc: 0.815509 + 0.000704201
[900]	cv_agg's auc: 0.816163 + 0.000694214
[950]	cv_agg's auc: 0.816732 + 0.000693815
[1000]	cv_agg's auc: 0.81727 + 0.000689625
[1050]	cv_agg's auc: 0.81775 + 0.000698459
[1100]	cv_agg's auc: 0.818221 + 0.00070402
[1150]	cv_agg's auc: 0.818689 + 0.000714516
[1200]	cv_agg's 

[9450]	cv_agg's auc: 0.832299 + 0.0005699
[9500]	cv_agg's auc: 0.832319 + 0.000571963
[9550]	cv_agg's auc: 0.832342 + 0.000572266
[9600]	cv_agg's auc: 0.832362 + 0.000572407
[9650]	cv_agg's auc: 0.832381 + 0.00057323
[9700]	cv_agg's auc: 0.832404 + 0.00057493
[9750]	cv_agg's auc: 0.832426 + 0.00057494
[9800]	cv_agg's auc: 0.832446 + 0.000571695
[9850]	cv_agg's auc: 0.832466 + 0.000572159
[9900]	cv_agg's auc: 0.832482 + 0.00056866
[9950]	cv_agg's auc: 0.832501 + 0.000570098
[10000]	cv_agg's auc: 0.83252 + 0.00057306
best n_estimators: 10000
best cv score: 0.8325197156888988


## 3.3.3 hyperopt自动化调参

* Epoch 1: TPE using 213/213 trials with best loss 0.166886
* Epoch 2: TPE using 24/24 trials with best loss 0.190400 ('dart', lr=0.01)
* Epoch 3: TPE using 31/31 trials with best loss 0.187324 ('dart', lr=0.01)
* Epoch 4: TPE using 30/30 trials with best loss 0.186235 ('dart', lr=0.01)
* Epoch 5: TPE using 20/20 trials with best loss 0.188895 ('dart', lr=0.005)
* Epoch 6: TPE using 35/35 trials with best loss 0.188685 ('dart', lr=0.005)

|param_name|epoch 1|epoch 2|epoch 3|epoch 4|epoch 5|epoch 6|final|
|:--|:--|:--|:--|:--|:--|:--|:--|
|boosting_type|'dart'|'dart'|'dart'|'dart'|'dart'|'dart'|'dart'|
|learning_rate|0.1514|0.01|0.01|0.01|0.005|0.005|0.005|
|max_depth|14|19|21|25|33|44|No|
|num_leaves|135|300|590|890|965|1130|1100|
|min_child_samples|288|330|250|200|160|110|105|
|min_child_weight|0.0048|0.0010|No|No|No|No|No|
|min_split_gain|0.9898|0.2010|No|No|No|No|No|

On ('dart', lr=0.5, n_estimators=200)
* Epoch 1: TPE using 33/33 trials with best loss 0.162092
* Epoch 2: TPE using 10/10 trials with best loss 0.162055
* Epoch 3: TPE using 8/8 trials with best loss 0.162175
* Epoch 4: TPE using 23/23 trials with best loss 0.161951
* Epoch 5: TPE using 15/15 trials with best loss 0.161962
* Epoch 6: TPE using 45/45 trials with best loss 0.161878

|param_name|epoch 1|epoch 2|epoch 3|epoch 4|epoch 5|final|
|:--|:--|:--|:--|:--|:--|:--|
|max_depth|20|24|22|20|20|20|
|num_leaves|200|200|180|180|190|190|
|min_child_samples|220|160|220|210|210|210|
|min_child_weight|0.056|0.045|0.045|0.055|0.055|0.055|
|feature_fraction|0.7|0.7|0.65|0.6|0.6|0.7|
|reg_alpha|0.001|0.029|0.001|0.009|0.005|0.005|
|reg_lambda|13.5|15.5|15|16|18|21|

In [4]:
X_train_temp, y_train_temp = X_train_all[int(0.2*(len(X_train_all))):], y_train_all[int(0.2*(len(X_train_all))):]
train_data = lgb.Dataset(X_train_temp, y_train_temp)
del X_train_all, y_train_all

In [10]:
import hyperopt
from hyperopt import hp

def hyperopt_objective(params):
    print('params:', params)
    model_lgb = LGBMClassifier(boosting_type='dart',
                               objective='binary', 
                               learning_rate=0.5,
                               n_estimators=200, 
                               max_depth=int(params['max_depth']),
                               num_leaves=int(params['num_leaves']),
                               min_child_samples=int(params['min_child_samples']),
                               min_child_weight=params['min_child_weight'],
                               feature_fraction = params['feature_fraction'],
#                                bagging_fraction = params['bagging_fraction'],
#                                bagging_freq = int(params['bagging_freq']),
                               reg_alpha = params['reg_alpha'],
                               reg_lambda = params['reg_lambda'],
#                                cat_smooth = int(params['cat_smooth']),
#                                bagging_fraction = 0.9,
#                                bagging_freq = 3,
#                                reg_alpha = 0.001,
#                                reg_lambda = 8,
#                                cat_smooth = 0,
                               n_jobs=16,
                               seed=666, 
                               silent=True,
                               device='gpu',)
    cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                    nfold=3, metrics=['logloss', 'auc'], verbose_eval=50,
                    early_stopping_rounds=10, stratified=False, shuffle=True,)
#     print('score:', 1 - max(cv_res['auc-mean']))
    return 1 - cv_res['auc-mean'][-1]  # 返回需要最小化的值

params_space = {
    'max_depth': 20,
    'num_leaves': 190,
    'min_child_samples': 210,
    'min_child_weight': 0.055,
    'feature_fraction': hp.quniform('feature_fraction', 0.6, 0.7, 0.1),
#     'bagging_fraction': hp.quniform('bagging_fraction', 0.5, 1, 0.1),
#     'bagging_freq': hp.quniform('bagging_freq', 1, 7, 1),
    'reg_alpha': hp.choice('reg_alpha', [0.001, 0.005, 0.01]),
    'reg_lambda': hp.quniform('reg_lambda', 21, 30, 1),
#     'cat_smooth': hp.quniform('cat_smooth', 0, 30, 10),
}  # 待优化的参数表

trials = hyperopt.Trials()

for _ in range(500):
    best = hyperopt.fmin(hyperopt_objective,
                         space=params_space,
                         algo=hyperopt.tpe.suggest,
                         max_evals=len(trials) + 1,
                         trials=trials,)
    print('best:', best)

print('\nfinal best:\n', best)

  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

[2019-12-11 21:02:08,014] INFO in tpe: tpe_transform took 0.001958 seconds
[2019-12-11 21:02:08,016] INFO in tpe: TPE using 0 trials


params:                                            
{'feature_fraction': 0.7000000000000001, 'max_depth': 20, 'min_child_samples': 220.0, 'min_child_weight': 0.055, 'num_leaves': 190, 'reg_alpha': 0.01, 'reg_lambda': 20.0}
[50]	cv_agg's auc: 0.834131 + 0.000562343          
[100]	cv_agg's auc: 0.836353 + 0.000514217         
[150]	cv_agg's auc: 0.837259 + 0.000479749         
[200]	cv_agg's auc: 0.837999 + 0.000443457         
100%|██████████| 1/1 [03:17<00:00, 197.57s/it, best loss: 0.16200127944160725]
best: {'feature_fraction': 0.7000000000000001, 'min_child_samples': 220.0, 'reg_alpha': 2, 'reg_lambda': 20.0}
  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

[2019-12-11 21:05:25,587] INFO in tpe: tpe_transform took 0.001520 seconds
[2019-12-11 21:05:25,588] INFO in tpe: TPE using 1/1 trials with best loss 0.162001


params:                                            
{'feature_fraction': 0.6000000000000001, 'max_depth': 20, 'min_child_samples': 220.0, 'min_child_weight': 0.055, 'num_leaves': 190, 'reg_alpha': 0.01, 'reg_lambda': 19.0}
[50]	cv_agg's auc: 0.834362 + 0.000405069          
[100]	cv_agg's auc: 0.836441 + 0.000468216         
[150]	cv_agg's auc: 0.837302 + 0.000293499         
[200]	cv_agg's auc: 0.837879 + 0.000257081         
100%|██████████| 1/1 [03:08<00:00, 188.23s/it, best loss: 0.16200127944160725]
best: {'feature_fraction': 0.7000000000000001, 'min_child_samples': 220.0, 'reg_alpha': 2, 'reg_lambda': 20.0}
  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

...

[2019-12-11 23:30:56,723] INFO in tpe: tpe_transform took 0.001591 seconds
[2019-12-11 23:30:56,724] INFO in tpe: TPE using 44/44 trials with best loss 0.161878


params:                                            
{'feature_fraction': 0.7000000000000001, 'max_depth': 20, 'min_child_samples': 210.0, 'min_child_weight': 0.055, 'num_leaves': 190, 'reg_alpha': 0.005, 'reg_lambda': 19.0}
[50]	cv_agg's auc: 0.834264 + 0.00020277           
[100]	cv_agg's auc: 0.83615 + 0.000209113          
[150]	cv_agg's auc: 0.837109 + 0.000252909         
[200]	cv_agg's auc: 0.837876 + 0.000281242         
100%|██████████| 1/1 [03:05<00:00, 185.23s/it, best loss: 0.1618777009087563]
best: {'feature_fraction': 0.7000000000000001, 'min_child_samples': 210.0, 'reg_alpha': 2, 'reg_lambda': 21.0}
  0%|          | 0/1 [00:00<?, ?it/s, best loss: ?]

[2019-12-11 23:34:01,962] INFO in tpe: tpe_transform took 0.004167 seconds
[2019-12-11 23:34:01,964] INFO in tpe: TPE using 45/45 trials with best loss 0.161878


params:                                            
{'feature_fraction': 0.6000000000000001, 'max_depth': 20, 'min_child_samples': 210.0, 'min_child_weight': 0.055, 'num_leaves': 190, 'reg_alpha': 0.01, 'reg_lambda': 22.0}
[50]	cv_agg's auc: 0.834317 + 0.00055688           
  0%|          | 0/1 [01:17<?, ?it/s, best loss: ?]


KeyboardInterrupt: 

# 4 calculate result

In [25]:
# 恢复数据
test = read_param('test')

## 4.1 LGBM结果

In [7]:
sub['label'] = model_lgb.predict_proba(test[feature_cols])[:, 1]

In [8]:
sub.to_csv('./storage/result.txt', index=None, header=None, sep='\t')

## 4.2 神经网络结果

In [37]:
res = model.predict(test[feature_cols], batch_size=BATCH_SIZE)
res

array([[0.51117074],
       [0.51117074],
       [0.51117074],
       ...,
       [0.5111708 ],
       [0.5111708 ],
       [0.5111708 ]], dtype=float32)

In [87]:
test[feature_cols]

,freq,freq_count,gender,gender_count,hour,q_ans_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_sum,q_has_img_max,...,uf_c2_count,uf_c3,uf_c3_count,uf_c4,uf_c4_count,uf_c5,uf_c5_count,uid_enc,uid_enc_count,wk
0,6,0.325453,4,0.0000,9,NaN,NaN,NaN,NaN,NaN,...,0.439214,203,0.479449,1376,0.000038,3,0.346068,1752473,0.000000,6
1,2,0.726702,4,0.0000,22,NaN,NaN,NaN,NaN,NaN,...,0.439214,396,0.456703,521,0.008534,3,0.346068,767938,0.000000,1
2,6,0.325453,2,0.4874,15,NaN,NaN,NaN,NaN,NaN,...,0.439214,205,0.083870,445,0.007599,3,0.346068,1147981,0.000009,3
3,5,1.000000,3,1.0000,4,4.0,11.0,3.0,12.0,0.0,...,0.439214,188,0.114102,1038,0.001950,2,1.000000,1372679,0.000015,2
4,6,0.325453,2,0.4874,19,NaN,NaN,NaN,NaN,NaN,...,0.168475,336,0.137013,788,0.001688,3,0.346068,1,0.297591,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141678,6,0.325453,4,0.0000,20,NaN,NaN,NaN,NaN,NaN,...,0.168475,81,0.020067,760,0.434533,3,0.346068,504330,0.000000,5
1141679,3,0.118357,4,0.0000,21,NaN,NaN,NaN,NaN,NaN,...,0.439214,121,0.094249,760,0.434533,3,0.346068,1066119,0.000003,1
1141680,2,0.726702,4,0.0000,15,NaN,NaN,NaN,NaN,NaN,...,0.001308,313,0.056923,760,0.434533,3,0.346068,435862,0.000000,0
1141681,6,0.325453,4,0.0000,8,NaN,NaN,NaN,NaN,NaN,...,0.439214,8,0.442846,1375,0.031122,3,0.346068,1287652,0.000015,0
